<a href="https://colab.research.google.com/github/AdityaVerma2809/NLP/blob/main/Word2Vec_embedding.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install gensim
!pip install python-Levenshtein

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.9/27.9 MB 60.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 153.3/153.3 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 68.9 MB/s eta 0:00:00


In [2]:
import gensim
import pandas as pd
import json

In [5]:
def load_clean_json(filename):
    """Reads a JSON Lines file, skipping lines with decoding errors."""

    # Counter for malformed lines
    malformed_count = 0

    with open(filename, 'r', encoding='utf-8') as f:
        for line in f:
            try:
                # Use standard json.loads to parse the line
                yield json.loads(line)
            except json.JSONDecodeError:
                # Increment counter and skip the bad line
                malformed_count += 1
                continue

    if malformed_count > 0:
        print(f"Warning: Skipped {malformed_count} malformed line(s) due to JSONDecodeError.")

# --- Run the Fix ---

# Load the clean data into a list of dictionaries
data_list = list(load_clean_json("amazon55.json"))

# Create the DataFrame
df = pd.DataFrame(data_list)

print(f"\nSuccessfully loaded DataFrame with {len(df)} rows.")


Successfully loaded DataFrame with 13827 rows.


In [6]:
df.head(5)

,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime
0,A30TL5EWN6DFXT,120401325X,christina,"[0, 0]",They look good and stick good! I just don't li...,4.0,Looks Good,1400630400,"05 21, 2014"
1,ASY55RVNIL0UD,120401325X,emily l.,"[0, 0]",These stickers work like the review says they ...,5.0,Really great product.,1389657600,"01 14, 2014"
2,A2TMXE2AFO7ONB,120401325X,Erica,"[0, 0]",These are awesome and make my phone look so st...,5.0,LOVE LOVE LOVE,1403740800,"06 26, 2014"
3,AWJ0WZQYMYFQ4,120401325X,JM,"[4, 4]",Item arrived in great time and was in perfect ...,4.0,Cute!,1382313600,"10 21, 2013"
4,ATX7CZYFXI1KW,120401325X,patrice m rogoza,"[2, 3]","awesome! stays on, and looks great. can be use...",5.0,leopard home button sticker for iphone 4s,1359849600,"02 3, 2013"


In [7]:
df.shape

(13827, 9)

In [8]:
df.reviewText[0]

"They look good and stick good! I just don't like the rounded shape because I was always bumping it and Siri kept popping up and it was irritating. I just won't buy a product like this again"

In [9]:
review_text=df.reviewText.apply(gensim.utils.simple_preprocess)
review_text.head(5)

,reviewText
0,"[they, look, good, and, stick, good, just, don..."
1,"[these, stickers, work, like, the, review, say..."
2,"[these, are, awesome, and, make, my, phone, lo..."
3,"[item, arrived, in, great, time, and, was, in,..."
4,"[awesome, stays, on, and, looks, great, can, b..."


In [10]:
model=gensim.models.Word2Vec(
    vector_size=100,
    window=5,
    min_count=2,
    workers=4)

In [11]:
model.build_vocab(review_text, progress_per=1000)

In [12]:
model.epochs

5

In [13]:
model.corpus_count

13827

In [14]:
model.train(review_text, total_examples=model.corpus_count, epochs=model.epochs)

(5280476, 7160615)

In [15]:
model.save("amazon_review_word2vec.model")

In [16]:
model.wv.most_similar("bad")

[('good', 0.6252102851867676),
 ('big', 0.5920971632003784),
 ('low', 0.5708891153335571),
 ('terrible', 0.5697025060653687),
 ('cheap', 0.5454312562942505),
 ('breaker', 0.5409548878669739),
 ('positive', 0.526506781578064),
 ('true', 0.526486873626709),
 ('recommeded', 0.5195582509040833),
 ('ok', 0.5180432200431824)]

In [17]:
model.wv.similarity(w1="good", w2="great")

np.float32(0.7581971)